In [1]:
import pymysql
import pandas as pd
#這個cell是我為你們寫好的function

def Sent_Data_To_Mysql(tagged_article, source_name, tag_type,keyword):
    db = pymysql.connect(host="18.217.252.187",port=3306, user="public_author",passwd="antimoneylaunderingisgood2",db="AML_News" ,charset='utf8')
    content = db.escape_string(tagged_article)
    try:
        with db.cursor() as cursor:
            sql0 = "SELECT `id` FROM `"+source_name+"` WHERE '"+tagged_article+"' LIKE CONCAT('%', `tagged_article`, '%')"
            cursor.execute(sql0)
            duplicate_id = cursor.fetchone()
            if(duplicate_id is not None):#如果已經存在一個content重複的
                sql1 = " INSERT INTO `"+source_name+"` (`id`, `tagged_article`,`tag_type`,`keyword`) VALUES (%s,%s,%s,%s) "
                sql2 = " ON DUPLICATE KEY UPDATE `tagged_article`= VALUES(`tagged_article`),`tag_type`= VALUES(`tag_type`),`keyword`= VALUES(`keyword`)"
                cursor.execute(sql1+sql2, (str(duplicate_id[0]), tagged_article,tag_type,keyword))
                print(str(duplicate_id[0]))
            else:#如果沒有重複的
                sql = "INSERT INTO `"+source_name+"`(`tagged_article`,`tag_type`,`keyword`) VALUES ( %s,%s,%s)"
                cursor.execute(sql, (tagged_article,tag_type,keyword))
        db.commit()
    finally:
        db.close()

def Get_Data_From_Mysql(source_name,keyword):
    contents_list=[]
    target_ids =[]
    db = pymysql.connect(host="18.217.252.187",port=3306, user="public_author",passwd="antimoneylaunderingisgood2",db="AML_News" ,charset='utf8')
    try:
        with db.cursor() as cursor:
            if(keyword=="all"):
                sql0 = "SELECT * FROM `"+source_name+"`"
                cursor.execute(sql0)
                contents_list = cursor.fetchall()
            else: 
                sql0 = "SELECT * FROM `"+source_name+"` WHERE '"+keyword+"' LIKE CONCAT('%', `keyword`)"
                cursor.execute(sql0)
                contents_list = cursor.fetchall()
                
        db.commit()
    finally:
        db.close()
    return  pd.DataFrame(list(contents_list), columns=['id','tagged_article','tag_type','keyword'])

In [2]:
import re
def find_sentence_and_nameIdx(tagged_article):
    sentence_and_nameIdx = []
    tagged_article = tagged_article.replace('\n', '').replace(' ', '').replace('。', '。\a').replace('《', '').replace('》', '').split('\a')
    for tagged_sentence in tagged_article:
        if tagged_sentence.find('/nr') < 0:
            continue
        sentence = tagged_sentence.replace('/nr', '').replace('/nt', '').replace('/ns', '').replace('/o', '')
        sentences = tagged_sentence
        name_list = []
        for findIter in re.finditer('/nr', tagged_sentence):
            start_idx = tagged_sentence[:findIter.start()].rfind('/o') + 2
            end_idx = findIter.start()
            name = tagged_sentence[start_idx:end_idx]
            if name not in name_list:
                name_list.append(name)
        names_and_idxs = []
        for name in name_list:
            name_and_idx = [name]
            for findIter in re.finditer(name, sentence):
                name_and_idx.append([findIter.start(), findIter.end()])
            names_and_idxs.append(name_and_idx)
        sentence_and_nameIdx.append([sentences, sentence, names_and_idxs])
    return sentence_and_nameIdx

def keep_name_right(sentence):
    """
    用tag過NER的句子斷句
    再把每一個斷詞送進結巴切詞
    希望盡可能保持姓名完整
    EX:
    input: 阮/nr 姓女子為首的/o 偽鈔集團/nt ，該集團今年二月已被/o 雲林/ns 警方偵破，
            起出新台幣千元假鈔二百六十萬元，但已有三十五萬元流入市面。警方陸續逮捕十二名涉案嫌犯，包括兩名在學校/o 福利社/nt
    
    用NER斷句 >> 阮, 姓女子為首的, 偽鈔集團, ，該集團今年二月已被,...
    
    ouput(用結巴斷詞) >> 阮, 姓, 女子, 為首, 的, 偽鈔, 集團, ... 
    """
    Sentence = []
    word_list = []
    temps = sentence.replace('nt',' ').replace('nr',' ').replace('o',' ')
    temps = temps.split(' ')
    for te in temps:
        Sentence.append(str(te).replace('/',''))
    #tempss = temps.replace('/','')
    Sentence
    for t in Sentence:
        to_convert= str(t)
        cc = OpenCC('t2s')  # (Optional )convert from Simplified Chinese to Traditional Chinese
        content = cc.convert(to_convert)#一樣轉成簡體字，這樣Stanford的系統才會比較準確
        #res = segmenter.segment(content)
        word_list.append(list(jieba.cut(content, cut_all=False)))
        #word_list.append(list(segmenter.segment(content)))
        #temps.split('/')
    word_list = sum(word_list,[])
    return(word_list)

def tag_POS(tree):
    """
    忽略這個
    """
    POS_tag = []
    #tree = tree.pos()
    for item in tree:
        word,POS = str(item).replace('(','').replace(')','').replace("'",'').split(',')
        POS_tag.append([word,POS])
    return POS_tag

In [3]:
import re
def find_sentence_and_nameIdx(tagged_article):
    sentence_and_nameIdx = []
    tagged_article = tagged_article.replace('\n', '').replace(' ', '').replace('。', '。\a').replace('《', '').replace('》', '').split('\a')
    for tagged_sentence in tagged_article:
        if tagged_sentence.find('/nr') < 0:
            continue
        sentence = tagged_sentence.replace('/nr', '').replace('/nt', '').replace('/ns', '').replace('/o', '')
        sentences = tagged_sentence
        name_list = []
        for findIter in re.finditer('/nr', tagged_sentence):
            start_idx = tagged_sentence[:findIter.start()].rfind('/o') + 2
            end_idx = findIter.start()
            name = tagged_sentence[start_idx:end_idx]
            if name not in name_list:
                name_list.append(name)
        names_and_idxs = []
        for name in name_list:
            name_and_idx = [name]
            for findIter in re.finditer(name, sentence):
                name_and_idx.append([findIter.start(), findIter.end()])
            names_and_idxs.append(name_and_idx)
        sentence_and_nameIdx.append([sentences, sentence, names_and_idxs])
    return sentence_and_nameIdx

def keep_name_right(sentence):
    """
    用tag過NER的句子斷句
    再把每一個斷詞送進結巴切詞
    希望盡可能保持姓名完整
    EX:
    input: 阮/nr 姓女子為首的/o 偽鈔集團/nt ，該集團今年二月已被/o 雲林/ns 警方偵破，
            起出新台幣千元假鈔二百六十萬元，但已有三十五萬元流入市面。警方陸續逮捕十二名涉案嫌犯，包括兩名在學校/o 福利社/nt
    
    用NER斷句 >> 阮, 姓女子為首的, 偽鈔集團, ，該集團今年二月已被,...
    
    ouput(用結巴斷詞) >> 阮, 姓, 女子, 為首, 的, 偽鈔, 集團, ... 
    """
    Sentence = []
    word_list = []
    temps = sentence.replace('nt',' ').replace('nr',' ').replace('o',' ')
    temps = temps.split(' ')
    for te in temps:
        Sentence.append(str(te).replace('/',''))
    #tempss = temps.replace('/','')
    Sentence
    for t in Sentence:
        to_convert= str(t)
        cc = OpenCC('t2s')  # (Optional )convert from Simplified Chinese to Traditional Chinese
        content = cc.convert(to_convert)#一樣轉成簡體字，這樣Stanford的系統才會比較準確
        #res = segmenter.segment(content)
        word_list.append(list(jieba.cut(content, cut_all=False)))
        #word_list.append(list(segmenter.segment(content)))
        #temps.split('/')
    word_list = sum(word_list,[])
    return(word_list)

def tag_POS(tree):
    """
    忽略這個
    """
    POS_tag = []
    #tree = tree.pos()
    for item in tree:
        word,POS = str(item).replace('(','').replace(')','').replace("'",'').split(',')
        POS_tag.append([word,POS])
    return POS_tag

In [4]:
"""
跑文法樹前一定要跑這個!!!
"""
#結巴斷詞、NLTK文法樹
import os
import jieba
from opencc import OpenCC
from nltk.parse.stanford import StanfordParser
from nltk.tokenize import StanfordSegmenter
from nltk.tag import StanfordNERTagger
from nltk.tag import StanfordPOSTagger
#這是比較直覺(攏長)的用法，直接指定路徑
chi_tagger = StanfordPOSTagger('./StanfordNLP/models/chinese-distsim.tagger',
                               './StanfordNLP/jars/stanford-ner.jar')
segmenter = StanfordSegmenter(
    java_class='edu.stanford.nlp.ie.crf.CRFClassifier',
    path_to_jar="./StanfordNLP/jars/stanford-segmenter-3.9.2.jar",
    path_to_slf4j="./StanfordNLP/jars/slf4j-api.jar",
    path_to_sihan_corpora_dict="./StanfordNLP/stanford-segmenter-2018-10-16/data",
    path_to_model="./StanfordNLP/stanford-segmenter-2018-10-16/data/pku.gz",
    path_to_dict="./StanfordNLP/stanford-segmenter-2018-10-16/data/dict-chris6.ser.gz"
)

os.environ["JAVA_HOME"] = "C:/Program Files/Java/jdk-12.0.1"#注意這邊你們電腦要安裝java jdk，並放入你們自己的jdk的安裝路徑
os.environ["CLASSPATH"] = ".\StanfordNLP\stanford-parser-2018-10-17"
os.environ["STANFORD_MODELS"] = "./StanfordNLP/models"

C:\Users\smile\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: DeprecationWarning: 
The StanfordTokenizer will be deprecated in version 3.2.5.
Please use nltk.parse.corenlp.CoreNLPTokenizer instead.'


# 建立文法樹並儲存

In [5]:
import pickle
def create_tree_and_save_it(i, crime_name):
    #跑文法樹
        #sentences: 保有NER tag
        #sentence: 原型句子
        #name_and_idx: 姓名、位置
    for sentences, sentence, name_and_idx in i:
        if crime_name in sentence: #如果犯罪名稱有在句子裡
            if len(sentence) < 500:      #如果犯罪名稱長度小於500
                word_list = keep_name_right(sentences) 
                #print(word_list)
                ch_parser = StanfordParser(model_path='./StanfordNLP/models/chinesePCFG.ser.gz')
                sent = ch_parser.parse(word_list) #形成文法樹
                pickle.dump(sent, file) #用pickle存
                #for sen in sent:
                    #tree_list = sen.pos() 
    return print('-----')

### 舉例: 要存蘋果新聞裡與洗錢有關的 (我只跑了database[0:10])

In [6]:
source_name = ['apple_daily','china_times', 'et_today', 'liberty_times', 'united_daily', 'judicial_gov']
output_dataframe = Get_Data_From_Mysql(source_name[0],'all') #洗錢為例
crime_name = ['人口販運', '性剝削', '偽造貨幣', '殺人', '重傷害', '搶奪' , '勒索' , '海盜', '恐怖主義', '資恐', '非法販運武器'
              , '贓物', '竊盜', '綁架', '拘禁', '妨礙自由', '環保犯罪', '偽造文書', '仿冒', '侵害商業秘密', '毒品犯運', '詐欺'
              , '走私', '稅務犯罪', '組織犯罪', '證卷犯罪', '貪汙賄賂', '洗錢']
#篩選句子內有犯罪名稱的句子
crime = (output_dataframe['keyword'] == crime_name[27]) #####這裡要改，(舉洗錢為例)
database = []
for art in output_dataframe[crime]["tagged_article"]:
    database.append(find_sentence_and_nameIdx(art)) #找出姓名，連成list
#畫文法樹並存在pickle
file = open('tree.pickle', 'wb')
for i in database[0:10]: #####可以像我先跑個幾筆([0:10])試一下，再跑全部，全部就是直接跑 >> for i in database
    create_tree_and_save_it(i, crime_name[27])   #####這裡要改，(舉洗錢為例)
file.close()

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\smile\AppData\Local\Temp\jieba.cache


-----
-----


Loading model cost 0.797 seconds.
Prefix dict has been built succesfully.


-----
-----
-----
-----
-----
-----
-----
-----


### 讀取tree, 轉成tag

In [7]:
tree_POS = []
with open('tree.pickle', 'rb') as file:
    while 1:
        try:
            tree = pickle.load(file)
            for i in tree:
                tree_POS.append(list(i.pos()))
        except:
            print("讀取完畢")
            break
file.close()

讀取完畢


In [8]:
for item in tree_POS:
    print(item)
    print('------')

[('研讨会', 'NN'), ('由', 'P'), ('法务部', 'NN'), ('检察', 'NN'), ('司', 'NN'), ('副司长', 'NN'), ('兼', 'CC'), ('洗钱', 'NN'), ('防制', 'NN'), ('办公室', 'NN'), ('执行', 'NN'), ('秘书', 'NN'), ('余丽贞', 'NN'), ('与', 'CC'), ('调查局', 'NN'), ('长', 'NN'), ('蔡清祥', 'NN'), ('开幕', 'VV'), ('，', 'PU'), ('也', 'AD'), ('勉励', 'VV'), ('检察机关', 'NN'), ('等', 'ETC'), ('执法人员', 'NN'), ('，', 'PU'), ('善用', 'VV'), ('金融', 'NN'), ('情报', 'NN'), ('与', 'CC'), ('资金', 'NN'), ('调查', 'NN'), ('手段', 'NN'), ('，', 'PU'), ('持续', 'VV'), ('精进', 'NN'), ('犯罪', 'NN'), ('金', 'JJ'), ('流', 'NN'), ('侦查', 'NN'), ('技巧', 'NN'), ('，', 'PU'), ('落实', 'VV'), ('政府', 'NN'), ('（', 'PU'), ('全民', 'NN'), ('防制', 'NR'), ('洗钱', 'NN'), ('、', 'PU'), ('追查', 'NN'), ('犯罪', 'NN'), ('金', 'JJ'), ('流', 'NN'), ('）', 'PU'), ('的', 'DEG'), ('政策', 'NN'), ('，', 'PU'), ('以', 'P'), ('彻底', 'JJ'), ('打击', 'NN'), ('洗钱', 'NN'), ('、', 'PU'), ('资恐', 'NR'), ('及', 'CC'), ('相关', 'JJ'), ('犯罪活动', 'NN'), ('，', 'PU'), ('以及', 'CC'), ('加强', 'VV'), ('跨部门', 'JJ'), ('合作', 'NN'), ('，', 'PU'), ('因应', 'NN'), ('未